In [3]:
import itertools

# Step 1: CNF Conversion Functions
def eliminate_implies(expr):
    # Replace P ⇒ Q with ¬P ∨ Q
    if "=>" in expr:
        antecedent, consequent = expr.split("=>")
        return f"¬{antecedent.strip()} ∨ {consequent.strip()}"
    return expr

def move_negation_inward(expr):
    # Handle negations: apply De Morgan's laws and negation rules
    expr = expr.replace("¬(∀", "∃¬").replace("¬(∃", "∀¬")
    expr = expr.replace("¬(P ∧ Q)", "¬P ∨ ¬Q").replace("¬(P ∨ Q)", "¬P ∧ ¬Q")
    return expr

def skolemize(expr):
    # Replace ∃x P(x) with P(f(c)) (Skolemization)
    if "∃" in expr:
        variable = expr[2]  # Extract the variable after ∃
        return expr.replace(f"∃{variable}", "").replace(variable, f"f(c)")
    return expr

def drop_universal_quantifiers(expr):
    # Drop ∀ quantifiers as all variables are universally quantified
    return expr.replace("∀", "")

def convert_to_cnf(statements):
    clauses = []
    for statement in statements:
        statement = eliminate_implies(statement)
        statement = move_negation_inward(statement)
        statement = skolemize(statement)
        statement = drop_universal_quantifiers(statement)
        clauses.extend(statement.split("∧"))
    return [clause.strip().split("∨") for clause in clauses]

# Step 2: Resolution Logic
def unify(x, y, subst={}):
    if subst is None:
        return None
    elif x == y:
        return subst
    elif isinstance(x, str) and x.islower():
        return unify_var(x, y, subst)
    elif isinstance(y, str) and y.islower():
        return unify_var(y, x, subst)
    elif isinstance(x, list) and isinstance(y, list) and len(x) == len(y):
        return unify(x[1:], y[1:], unify(x[0], y[0], subst))
    else:
        return None

def unify_var(var, x, subst):
    if var in subst:
        return unify(subst[var], x, subst)
    elif x in subst:
        return unify(var, subst[x], subst)
    else:
        subst[var] = x
        return subst

def resolve(clause1, clause2):
    resolvents = []
    for literal1 in clause1:
        for literal2 in clause2:
            if literal1.startswith('¬') and literal1[1:] == literal2:
                resolvent = list(set(clause1) - {literal1}) + list(set(clause2) - {literal2})
                resolvents.append(resolvent)
            elif literal2.startswith('¬') and literal2[1:] == literal1:
                resolvent = list(set(clause1) - {literal1}) + list(set(clause2) - {literal2})
                resolvents.append(resolvent)
    return resolvents

def resolution(clauses):
    new_clauses = set()
    while True:
        pairs = list(itertools.combinations(clauses, 2))
        for (clause1, clause2) in pairs:
            resolvents = resolve(clause1, clause2)
            for resolvent in resolvents:
                if not resolvent:
                    print("Contradiction found!")
                    return True
                new_clauses.add(tuple(resolvent))

        if new_clauses.issubset(set(map(tuple, clauses))):
            print("No new clauses generated. Resolution complete.")
            return False

        for clause in new_clauses:
            clauses.append(list(clause))

# Step 3: Main Function
def main():
    # Input FOL statements
    statements = [
        "∀x (P(x) => Q(x))",
        "P(a)",
        "¬Q(a)"
    ]
    print("Original Statements:")
    for s in statements:
        print(s)

    # Convert to CNF
    clauses = convert_to_cnf(statements)
    print("\nClauses in CNF:")
    for clause in clauses:
        print(clause)

    # Apply Resolution
    result = resolution(clauses)
    if result:
        print("\nThe given set of clauses is unsatisfiable (proof completed).")
    else:
        print("\nThe given set of clauses is satisfiable.")

if __name__ == "__main__":
    main()

print("\n1bm22cs022")


Original Statements:
∀x (P(x) => Q(x))
P(a)
¬Q(a)

Clauses in CNF:
['¬x (P(x) ', ' Q(x))']
['P(a)']
['¬Q(a)']
No new clauses generated. Resolution complete.

The given set of clauses is satisfiable.

1bm22cs022
